# Sample Projects
This notebook is intended as a playground to try out various python widgets, ideas etc. This notebook is publically saved on github.

In [1]:
# tax rate is fixed
TAX_RATE = .0605

In [2]:
from math import floor

def round(x):
    return floor((x * 100) + .5)/100.0

class LineItem:
    def __init__(self,label,each_amount,qty):
        self.label = label
        self.each_amount = each_amount
        self.qty = qty
        self.line_amount = round(self.each_amount * self.qty)
    def __str__(self):
        if self.label == "LTAX":
            return "{} {} x {:0,.4f} = ${:0,.2f}".format(self.label,self.qty,self.each_amount,self.line_amount)
        return "{} {} x ${:0,.2f} = ${:0,.2f}".format(self.label,self.qty,self.each_amount,self.line_amount)

def invoice_str(invoice):
    return [str(i) for i in invoice]

In [3]:
def simple_prediction(stay_nights,gross_paid,tax_paid,cln_fee,pet_fee,npets,rate):
    print("Trying prediction[simple,pet={}]".format(npets))
    predicted_nightly_rate = round((gross_paid - tax_paid - cln_fee - (pet_fee * npets)) / stay_nights)
    rent = (predicted_nightly_rate * stay_nights)
    pets = pet_fee * npets
    subtotal = rent + cln_fee + pets
    tax_calc = round(subtotal * rate)
    total = subtotal + tax_calc
    print("rent={} pets={} subtotal={} total={}".format(rent,pets,subtotal,total))
    invoice = [
        LineItem("RNT",predicted_nightly_rate,stay_nights),
        LineItem("CLN",cln_fee,1),
        LineItem("PET",pet_fee,npets),
        LineItem("LTAX",rate,subtotal)]
    print("Trying invoice={}".format(invoice_str(invoice)))
    
    if total != gross_paid:
        print("Invoice FAILED!!")
        raise RuntimeError("Could not predict invoice!")
        
    print("Invoice succeeded.")
    return invoice

In [4]:
def predict_invoice(stay_nights,gross_paid,tax_paid,cln_fee=135.00,pet_fee=95.00,rate=TAX_RATE):
    """
    Predicts the invoice line items given stay nights, gross amount paid,
    taxes paid, cleaning fee, and pet fees.  If the invoice can not be predicted,
    raise an exception.
    
    Parameters
    ----------
      stay_nights : integer
          The number of nights stayed. Used to multiple the nightly rate to determine overal rent.
      gross_paid : double
          The amount (USD) paid for the stay
      tax_paid : double
          The amount (USD) paid in taxes.  This should be total amount (rent + fees) * TAX_RATE.
      cln_fee : double
          The amount (USD) charged/paid for cleaning fees.
      pet_fee : double
          The amount (USD) charged/paid for pet fees.
    
    Returns
    -------
       A list of invoice items or raises an exception if the invoice could not be created
    """
    
#    # initialize the list of predictions
#    # a prediction is just a function that tries to return a list of invoice items
#    predictions = [simple_prediction]

    print("Predicting invoice:")
    print("   (stay_nights :: gross_paid :: tax_paid) = ({} :: ${:0,.2f} :: ${:0,.2f})".format(stay_nights,gross_paid,tax_paid))
#    prediction = predictions.pop()
    invoice = []
    npets = 0
    while True:
        try:
#            invoice = prediction(stay_nights,gross_paid,tax_paid,cln_fee,pet_fee,npets)
            invoice = simple_prediction(stay_nights,gross_paid,tax_paid,cln_fee,pet_fee,npets,rate)
            break
        except RuntimeError:
#            try:
#                prediction = predictions.pop()
#            except IndexError:
#                raise RuntimeError("Could not predict invoice!")
            npets += 1
            if npets==3 and rate==.05:
                raise RuntimeError("Could not predict invoice!")
            elif npets==3:
                return predict_invoice(stay_nights,gross_paid,tax_paid,cln_fee,pet_fee,rate=.05)

    return invoice

In [5]:
import csv

with open('payments.tsv') as payments_file:
    values_tsv = csv.reader(payments_file, delimiter='\t')
    data=[row for row in values_tsv]
data

[['HA-9DHJ3M', '20201108', '7', '1283.21', '41.06', '1242.15', '73.21'],
 ['HA-GYM6PW', '20201121', '8', '1400.70', '44.82', '1355.88', '66.70'],
 ['HA-YL95KM', '20201125', '3', '685.65', '21.94', '663.71', '32.65']]

In [6]:
def invoice_label(row):
    return "{} co {}:{}".format(row[0],row[1],row[3])

def to_invoice(row):
    try:
        return invoice_str(predict_invoice(stay_nights=int(row[2]),
                                          gross_paid=float(row[3]),
                                          tax_paid=float(row[6])))
    except:
        return "INVOICE FAIL"

results = {invoice_label(row):to_invoice(row) for row in data}
results

Predicting invoice:
   (stay_nights :: gross_paid :: tax_paid) = (7 :: $1,283.21 :: $73.21)
Trying prediction[simple,pet=0]
rent=1074.99 pets=0.0 subtotal=1209.99 total=1283.19
Trying invoice=['RNT 7 x $153.57 = $1,074.99', 'CLN 1 x $135.00 = $135.00', 'PET 0 x $95.00 = $0.00', 'LTAX 1209.99 x 0.0605 = $73.20']
Invoice FAILED!!
Trying prediction[simple,pet=1]
rent=980.0 pets=95.0 subtotal=1210.0 total=1283.21
Trying invoice=['RNT 7 x $140.00 = $980.00', 'CLN 1 x $135.00 = $135.00', 'PET 1 x $95.00 = $95.00', 'LTAX 1210.0 x 0.0605 = $73.21']
Invoice succeeded.
Predicting invoice:
   (stay_nights :: gross_paid :: tax_paid) = (8 :: $1,400.70 :: $66.70)
Trying prediction[simple,pet=0]
rent=1199.04 pets=0.0 subtotal=1334.04 total=1414.75
Trying invoice=['RNT 8 x $149.88 = $1,199.04', 'CLN 1 x $135.00 = $135.00', 'PET 0 x $95.00 = $0.00', 'LTAX 1334.04 x 0.0605 = $80.71']
Invoice FAILED!!
Trying prediction[simple,pet=1]
rent=1104.0 pets=95.0 subtotal=1334.0 total=1414.71
Trying invoice=['RNT

{'HA-9DHJ3M co 20201108:1283.21': ['RNT 7 x $140.00 = $980.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 1 x $95.00 = $95.00',
  'LTAX 1210.0 x 0.0605 = $73.21'],
 'HA-GYM6PW co 20201121:1400.70': ['RNT 8 x $138.00 = $1,104.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 1 x $95.00 = $95.00',
  'LTAX 1334.0 x 0.0500 = $66.70'],
 'HA-YL95KM co 20201125:685.65': ['RNT 3 x $141.00 = $423.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 1 x $95.00 = $95.00',
  'LTAX 653.0 x 0.0500 = $32.65']}